In [1]:
%config InlineBackend.figure_format = 'svg'

In [8]:
import networkx as nx
#from node2vec import Node2Vec
import os
import pickle
import random
from itertools import chain
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from random import sample
import seaborn as sns
import numpy as np

## Functions

In [9]:
working_dir = "data/covid_literature/knowing01"

In [10]:
pat_prot = pd.read_csv(working_dir + '/annotate-out-kimonoGenes.PatientData-links.Lax-Proteins.simplified.txt', sep="\t",header=None )
pat_genes = pd.read_csv(working_dir + '/annotate-out-kimonoGenes.PatientData-links.Stringent-Genes.simplified.txt', sep="\t",header=None )

In [11]:
lit_genes =  []
lit_genes.append(pat_prot[0].tolist())
lit_genes.append(pat_genes[0].tolist())
lit_genes.sort()

In [12]:
unique_genes = {x for l in lit_genes for x in l}
len(unique_genes) #860

2573

In [8]:
unique_genes

{'CCHCR1',
 'CCR1',
 'CDC37',
 'FYCO1',
 'KANSL1',
 'KEAP1',
 'LIMD1',
 'LTF',
 'LZTFL1',
 'MAPT',
 'NSF',
 'OAS1',
 'OAS3',
 'RAVER1',
 'SACM1L',
 'TYK2'}

In [9]:
with open(working_dir + '/unique_covid_genes_covid_patient_protein.txt', 'w') as f:
    for val in unique_genes:
        f.write(val)
        f.write("\n")

In [28]:
covid_genes = pd.read_csv(working_dir + '/unique_covid_genes_covid_patient_protein.txt', names=['covidGenes'])

In [32]:
covid_genes = covid_genes.sort_values(by=['covidGenes'])

# read data frame

In [13]:
final_non_gene_connections = pd.read_csv("data/meanSim_all/meanSim_robust_nongenes.txt")

In [14]:
final_non_gene_connections[final_non_gene_connections['node1'].isin(unique_genes)]

,node1,node2,neighbour,type_node1,type_node2,N,mean_sim,max_sim,sem_sim
0,A2M,SMTSD_Heart_Atrial_Appendage,1,gene,tissue,100,0.663943,0.741950,0.021931
15,AATK,SMTSD_Spleen,1,gene,tissue,100,0.729235,0.780037,0.027303
16,AATK,SMTSD_Minor_Salivary_Gland,1,gene,tissue,99,0.690638,0.768442,0.034700
17,ABCA1,AGE,1,gene,pheno,100,0.727772,0.772100,0.022543
18,ABCB1,SMTSD_Colon_Transverse,1,gene,tissue,100,0.977041,0.985916,0.004753
...,...,...,...,...,...,...,...,...,...
6062,ZPR1,obesity,1,gene,prs,98,0.657555,0.701223,0.019795
6071,ZWILCH,SMTSD_Brain_Frontal_Cortex_BA9,1,gene,tissue,80,0.658528,0.749778,0.032053
6072,ZXDC,SMTSD_Brain_Nucleus_accumbens_basal_ganglia,1,gene,tissue,94,0.700148,0.767734,0.028239
6074,ZYX,MHPNMNIA,1,gene,como,100,0.685412,0.733330,0.019922


In [16]:
final_non_gene_connections["source"] = 'patient'

In [17]:
final_non_gene_connections

,node1,node2,neighbour,type_node1,type_node2,N,mean_sim,max_sim,sem_sim,source
0,A2M,SMTSD_Heart_Atrial_Appendage,1,gene,tissue,100,0.663943,0.741950,0.021931,patient
1,AAGAB,SMTSD_Brain_Frontal_Cortex_BA9,1,gene,tissue,81,0.711958,0.803277,0.051792,patient
2,AAK1,SMTSD_Brain_Anterior_cingulate_cortex_BA24,1,gene,tissue,100,0.757863,0.827443,0.034475,patient
3,AAK1,SMTSD_Breast_Mammary_Tissue,1,gene,tissue,100,0.668527,0.741021,0.039284,patient
4,AAK1,SMTSD_Skin_Not_Sun_Exposed_Suprapubic,1,gene,tissue,100,0.662176,0.722359,0.030489,patient
...,...,...,...,...,...,...,...,...,...,...
6071,ZWILCH,SMTSD_Brain_Frontal_Cortex_BA9,1,gene,tissue,80,0.658528,0.749778,0.032053,patient
6072,ZXDC,SMTSD_Brain_Nucleus_accumbens_basal_ganglia,1,gene,tissue,94,0.700148,0.767734,0.028239,patient
6073,ZYG11B,SMTSD_Brain_Substantia_nigra,3,gene,tissue,91,0.696453,0.783530,0.043163,patient
6074,ZYX,MHPNMNIA,1,gene,como,100,0.685412,0.733330,0.019922,patient


In [19]:
final_non_gene_connections.to_csv("results/covid_net/edgelist/patient_meanSim.csv")